In [12]:
import networkx as nx
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
import numpy as np
import math
import pickle
from tqdm import tqdm_notebook
from multiprocessing import Pool
from collections import defaultdict
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

## Parameters Setting

In [14]:
# Model settings
latent_dim = 300
epochs = 600
lr = 100
lambda_V = 1e-5
lambda_w = 1e-4
param_alpha = 1
focal_alpha = 0.1
focal_gamma = 1.5
edge_attrs = ['dim_1', 'dim_2', 'dim_3']
L = 20

# Graph fetching settings
weight_type = 'embed'  # [weight_none, weight_log, count_larger8000]
offset = 0

# To store all the infomation of dynamic graphs
dyngraph_info = defaultdict(dict)

## Fetch data by date

In [8]:
class DateYM:
    def __init__(self, year, month):
        self.year = year
        self.month = month - 1    # 0 ~ 11, from Jan to Dec
        
    def export_tuple(self):
        return (self.year, self.month+1)
    
    def add_year(self, y):
        self.year += y
        
    def substract_year(self, y):
        self.year -= y
        
    def add_month(self, m):
        self.month += m
        self.year += math.floor(self.month / 12)
        self.month = self.month % 12
        
    def subtract_month(self, m):
        self.month -= m
        tmp_year = math.floor(self.month / 12)
        self.year += tmp_year
        self.month += -tmp_year * 12
        
    def is_larger_than(self, ym):
        return self.year*12 + self.month > ym.year*12 + ym.month
    
    def is_smaller_than(slef, ym):
        return self.year*12 + self.month < ym.year*12 + ym.month
    
    def is_equal(self, ym):
        return self.year*12 + self.month == ym.year*12 + ym.month

    
def list_date_tuples(from_date, to_date):
    ret = []
    tmp_date = DateYM(*from_date.export_tuple())
    while not tmp_date.is_larger_than(to_date):
        ret.append(tmp_date.export_tuple())
        tmp_date.add_month(1)
    return ret


def fetch_data_by_month(date_ym, trans_offset=6):
    year, month = date_ym
    
    # Get view: WireTrans
    from_date = pd.to_datetime("{}/{}/{}".format(month, 1, year))
    to_date = from_date + pd.DateOffset(months=1)
    offset_date = from_date - pd.DateOffset(months=trans_offset)
    view_wiretrans = WireTrans[(WireTrans.trans_date > offset_date) & 
                               (WireTrans.trans_date < to_date)]
    
    # Get view: CustInfo
    view_customer = CustInfo[CustInfo.open_date < to_date]
    
    # Attach label onto CustInfo
    target_list = SARCase[(SARCase.Status_SAR == 4) & 
                          (SARCase.created_date > from_date) & 
                          (SARCase.created_date < to_date)]['customerno'].unique()
    # print ('# of SAR customers: {}'.format(len(target_list)))
    view_customer['label'] = view_customer.apply(lambda x: 1 if x['customerno'] in target_list else 0, axis=1)
    
    return view_wiretrans, view_customer

In [9]:
from_date_ym = DateYM(2018, 1)
to_date_ym = DateYM(2019, 6)
list_date_seq = list_date_tuples(from_date_ym, to_date_ym)
seq_length = len(list_date_seq)
print(list_date_seq)

[(2018, 1), (2018, 2), (2018, 3), (2018, 4), (2018, 5), (2018, 6), (2018, 7), (2018, 8), (2018, 9), (2018, 10), (2018, 11), (2018, 12), (2019, 1), (2019, 2), (2019, 3), (2019, 4), (2019, 5), (2019, 6)]


## Load Edge Embeddings

In [17]:
def get_edge_embed(embed_path, groupby_method='sum'):
    df = pd.read_csv(embed_path)
    df.columns = ['index', 'from_vertex', 'to_vertex', 'dim_1', 'dim_2', 'dim_3']
    df['vertex_index'] = df.apply(lambda x: str(set([x['from_vertex'], x['to_vertex']])), axis=1)
    if groupby_method == 'sum':
        df_new = df.groupby('vertex_index').sum().reset_index().reset_index()
    else:
        df_new = df.groupby('vertex_index').mean().reset_index().reset_index()
    df_new['from_vertex'] = df_new.apply(lambda x: x['vertex_index'][1:-1].split(',')[0][1:-1], axis=1)
    df_new['to_vertex'] = df_new.apply(lambda x: x['vertex_index'][1:-1].split(',')[1][2:-1], axis=1)
    
    return df_new

In [18]:
for year, month in list_date_seq:
    edge_embed_path = '../data/edge_embeddings/Edge_Embedd_{}-{}.csv'.format(year, month)
    edge_embed = get_edge_embed(edge_embed_path)
    dyngraph_info[(year, month)]['edge_embed'] = edge_embed

In [23]:
dyngraph_info[(2018, 1)]['edge_embed'].head()

,level_0,vertex_index,index,dim_1,dim_2,dim_3,from_vertex,to_vertex
0,0,"{'AA0AAA0BBXAJCSM=', 'AgQDCAMDIw=='}",1789,0.903,0.940,1.331,AA0AAA0BBXAJCSM=,AgQDCAMDIw==
1,1,"{'AA0AAAECDnAMCSM=', 'BgcFAgAHIw=='}",59163,0.868,0.963,1.338,AA0AAAECDnAMCSM=,BgcFAgAHIw==
2,2,"{'AA0AAAwFAHcLIw==', 'BAUABQcLIw=='}",11116,0.914,0.923,1.328,AA0AAAwFAHcLIw==,BAUABQcLIw==
3,3,"{'AA0AAAwFAHwBASM=', 'BgcACQIBIw=='}",46873,0.940,0.874,1.319,AA0AAAwFAHwBASM=,BgcACQIBIw==
4,4,"{'AA0AAAwFB3cNCSM=', 'BgUECQYGIw=='}",36907,1.320,1.430,0.942,AA0AAAwFB3cNCSM=,BgUECQYGIw==


## Vertex Indexing
To construct an indexing mapping between `entry_index` and `custimer_id`.

In [63]:
for year, month in list_date_seq:
    vertex_ids_t = set.union(set(dyngraph_info[(year, month)]['edge_embed']['from_vertex']),
                             set(dyngraph_info[(year, month)]['edge_embed']['to_vertex']))
    dict_int2key = dict(map(lambda x, y: (x, y), range(len(vertex_ids_t)), vertex_ids_t))
    dict_key2int = dict(map(lambda x, y: (y, x), range(len(vertex_ids_t)), vertex_ids_t))
    dyngraph_info[(year, month)]['int2key'] = dict_int2key
    dyngraph_info[(year, month)]['key2int'] = dict_key2int

In [ ]:
for year, month in list_date_seq:
    with open('../data/index_mapping/int2key_{}{}.pickle'.format(str(year), str(month).zfill(2)), 'wb') as handle:
        pickle.dump(dyngraph_info[(year, month)]['int2key'], handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('../data/index_mapping/key2int_{}{}.pickle'.format(str(year), str(month).zfill(2)), 'wb') as handle:
        pickle.dump(dyngraph_info[(year, month)]['key2int'], handle, protocol=pickle.HIGHEST_PROTOCOL)

## Load Ground Truth Labels
- `FILTER`: required for monitoring 
- `announce`: suspicious account for AML
- `Y_SAR`: AML account

In [118]:
for year, month in list_date_seq:
    ctbc_path = '../data/aml_all_{}{}_y.csv'.format(str(year)[2:], str(month).zfill(2))
    ctbc_data = pd.read_csv(ctbc_path)
    
    dict_key2int_t = dyngraph_info[(year, month)]['key2int']
    pos_index = [dict_key2int_t[x] for x in list(ctbc_data[ctbc_data['Y_SAR'] == 1]['customerno']) if x in dict_key2int_t]
    y_t = np.zeros((len(dict_key2int_t), 1))
    y_t[pos_index, 0] = 1
    dyngraph_info[(year, month)]['y'] = y_t

    print("({}, {}): # of FILTER = {}, # of announce = {}, # of Y_SAR = {}, # of PosVertex = {}".
          format(year, month, 
                 sum(ctbc_data['FILTER']), sum(ctbc_data['announce']), 
                 sum(ctbc_data['Y_SAR']), int(sum(y_t))))

(2018, 1): # of FILTER = 34238, # of announce = 353, # of Y_SAR = 15, # of PosVertex = 14
(2018, 2): # of FILTER = 34238, # of announce = 253, # of Y_SAR = 11, # of PosVertex = 11
(2018, 3): # of FILTER = 34238, # of announce = 223, # of Y_SAR = 16, # of PosVertex = 16
(2018, 4): # of FILTER = 34238, # of announce = 315, # of Y_SAR = 6, # of PosVertex = 6
(2018, 5): # of FILTER = 34238, # of announce = 341, # of Y_SAR = 9, # of PosVertex = 9
(2018, 6): # of FILTER = 34238, # of announce = 315, # of Y_SAR = 14, # of PosVertex = 14
(2018, 7): # of FILTER = 34238, # of announce = 355, # of Y_SAR = 36, # of PosVertex = 36
(2018, 8): # of FILTER = 34238, # of announce = 352, # of Y_SAR = 13, # of PosVertex = 13
(2018, 9): # of FILTER = 34238, # of announce = 307, # of Y_SAR = 9, # of PosVertex = 9
(2018, 10): # of FILTER = 34238, # of announce = 455, # of Y_SAR = 21, # of PosVertex = 21
(2018, 11): # of FILTER = 34238, # of announce = 430, # of Y_SAR = 17, # of PosVertex = 17
(2018, 12): # 

## Load Graph by Month

In [120]:
for year, month in list_date_seq:
    edge_embed_t = dyngraph_info[(year, month)]['edge_embed']
    key2int_t = dyngraph_info[(year, month)]['key2int']
    int2key_t = dyngraph_info[(year, month)]['int2key']
    G_t = nx.from_pandas_edgelist(edge_embed_t, 'from_vertex', 'to_vertex', 
                                  edge_attr=edge_attrs,
                                  create_using=nx.MultiGraph())
    G_t = nx.relabel_nodes(G_t, key2int_t)
    L_t = nx.normalized_laplacian_matrix(G_t, nodelist=int2key_t)
    A_t = nx.to_scipy_sparse_matrix(G_t, nodelist=int2key_t)
    A_ts = [nx.to_scipy_sparse_matrix(G_t, weight=attr, nodelist=int2key_t) for attr in edge_attrs]
    dyngraph_info[(year, month)]['graph_laplacian'] = L_t
    dyngraph_info[(year, month)]['graph_adjacent'] = A_ts
    print('({}, {}): Finished graph constructuion, |V| = {}'.format(year, month, A_t.shape[0]))

(2018, 1): Finished graph constructuion, |V| = 53870
(2018, 2): Finished graph constructuion, |V| = 53621
(2018, 3): Finished graph constructuion, |V| = 54041
(2018, 4): Finished graph constructuion, |V| = 55085
(2018, 5): Finished graph constructuion, |V| = 56137
(2018, 6): Finished graph constructuion, |V| = 56499
(2018, 7): Finished graph constructuion, |V| = 56850
(2018, 8): Finished graph constructuion, |V| = 57139
(2018, 9): Finished graph constructuion, |V| = 56367
(2018, 10): Finished graph constructuion, |V| = 56248
(2018, 11): Finished graph constructuion, |V| = 55286
(2018, 12): Finished graph constructuion, |V| = 55162
(2019, 1): Finished graph constructuion, |V| = 55141
(2019, 2): Finished graph constructuion, |V| = 52634
(2019, 3): Finished graph constructuion, |V| = 50845
(2019, 4): Finished graph constructuion, |V| = 49940
(2019, 5): Finished graph constructuion, |V| = 48803
(2019, 6): Finished graph constructuion, |V| = 47612


## Embedding Projection on a Dynamic graph

In [121]:
def sigmoid(x):
    if len(x.shape) > 0:
        x[x < -5] = -5.
    else:
        x = -5. if x < -5. else x
    return 1 / (1 + np.exp(-x))

In [144]:
def update_params(input_pkg):
    t, hetergraph = input_pkg
    L_t, A_ts = hetergraph
    date_ym = list_date_seq[t]
    
    key2int_t = dyngraph_info[date_ym]['key2int']
    int2key_t = dyngraph_info[date_ym]['int2key']
    
    y_t = dyngraph_info[date_ym]['y']
    V_t = dyngraph_info[date_ym]['vertex_embed']
    w1 = classifiers['w1']
    w2 = classifiers['w2']
    we = classifiers['we']
    theta = classifiers['theta']
    
    A_t = scipy.sparse.csr_matrix(A_ts[0].shape)
    for index_attr in range(we.shape[0]):
        tmpA_t = A_ts[index_attr]
        A_t += sigmoid(we[index_attr,0]) * tmpA_t
        
    if (t + 1) < seq_length:
        date_ym_plus = list_date_seq[t + 1]
        
        key2int_t_plus = dyngraph_info[date_ym_plus]['key2int']
        int2key_t_plus = dyngraph_info[date_ym_plus]['int2key']
        index_t_2_t_plus = [key2int_t_plus[x] if x in key2int_t_plus else -1 for x in int2key_t]
        
        # Get V_t_plus aligned in V_t index order
        V_t_plus_orig = dyngraph_info[date_ym_plus]['vertex_embed']
        V_t_plus_orig = np.append(V_t_plus_orig, np.zeros((1, V_t_plus_orig.shape[1])), axis=0)
        V_t_plus = V_t_plus_orig[index_t_2_t_plus]
        
        # Get y_t_plus aligned in y_t index order
        y_t_plus_orig = dyngraph_info[date_ym_plus]['y']
        y_t_plus_orig = np.append(y_t_plus_orig, np.zeros((1,1)), axis=0)
        y_t_plus = y_t_plus_orig[index_t_2_t_plus]
    else:
        V_t_plus = None
        y_t_plus = None
        key2int_t_plus = None
        int2key_t_plus = None

    if t > 0:
        date_ym_minus = list_date_seq[t - 1]
        
        key2int_t_minus = dyngraph_info[date_ym_minus]['key2int']
        int2key_t_minus = dyngraph_info[date_ym_minus]['int2key']
        index_t_2_t_minus = [key2int_t_minus[x] if x in key2int_t_minus else -1 for x in int2key_t]
        
        # Get V_t_plus aligned in V_t index order
        V_t_minus_orig = dyngraph_info[date_ym_minus]['vertex_embed']
        V_t_minus_orig = np.append(V_t_minus_orig, np.zeros((1, V_t_minus_orig.shape[1])), axis=0)
        V_t_minus = V_t_minus_orig[index_t_2_t_minus]
                
        y_t_minus_orig = dyngraph_info[date_ym_minus]['y']
        y_t_minus_orig = np.append(y_t_minus_orig, np.zeros((1,1)), axis=0)
        y_t_minus = y_t_minus_orig[index_t_2_t_minus] 
    else:
        V_t_minus = None
        y_t_minus = None
        key2int_t_minus = None
        int2key_t_minus = None
    
    # Create sparse matrix S
    vec_i, vec_j, vec_v = scipy.sparse.find(A_t)
    vec_data = vec_v * sigmoid(-np.sum(V_t[vec_i, :] * V_t[vec_j, :], axis=1))
    S_t = scipy.sparse.csr_matrix((vec_data, (vec_i, vec_j)), shape=A_t.shape)
    
    # Create sparse matrix R
    l = [[u]*L for u in range(A_t.shape[0])]
    smpl_i = [item for sublist in l for item in sublist]
    l = [list(np.random.choice(A_t.shape[0], min(L*A_t[u,:].nnz, A_t.shape[0]), replace=False)) for u in range(A_t.shape[0])]
    smpl_j = [item for sublist in l for item in sublist]
    smpl_index = list(set(zip(smpl_i, smpl_j)) | set(zip(vec_i, vec_j)))
    smpl_i, smpl_j = zip(*smpl_index)
    smpl_data = sigmoid(np.sum(V_t[smpl_i, :] * V_t[smpl_j, :], axis=1))
    R_t = scipy.sparse.csr_matrix((smpl_data, (smpl_i, smpl_j)), shape=A_t.shape)
    
    # Compute normalization term
    norm_graph = 1 / (vec_data.shape[0] + smpl_data.shape[0])
    norm_laplacian = 1 / (V_t.shape[0] * V_t.shape[1])
    norm_time = 1 / (V_t.shape[0] * V_t.shape[1])
    norm_pred = 10 / V_t.shape[0]
    
    # Update variables
    for itr in range(3):
        # Create vector z_t
        vec_one = np.ones((V_t.shape[0], 1))
        tmp_recur = np.cos(theta) * V_t + np.sin(theta) * V_t_minus if V_t_minus is not None else V_t
        tmp = np.dot(tmp_recur, w1) + 0.5 * np.dot(tmp_recur * tmp_recur, w2)
        pred_t = sigmoid(tmp)
        q_t = (vec_one - y_t) * (vec_one - pred_t) + y_t * pred_t + 1e-10
        q_t[q_t > 1] = 1.
        z_t = focal_alpha * (-focal_gamma * (1 - q_t) ** (focal_gamma - 1) * np.log(q_t) + \
                             1/q_t * (1 - q_t) ** focal_gamma) * \
              (2 * y_t - vec_one) * pred_t * (vec_one - pred_t)
        
        # Compute the gradient w.r.t. V_t
        grad_V_t = norm_graph * (-param_alpha * S_t * V_t + R_t * V_t) + \
                   lambda_V * V_t + \
                   norm_laplacian * L_t * L_t * V_t
        
        if V_t_plus is not None:
            tmp_recur = np.cos(theta) * V_t_plus + np.sin(theta) * V_t
            pred_t_plus = sigmoid(np.dot(tmp_recur, w1) + 0.5 * np.dot(tmp_recur**2, w2))
            q_t_plus = (vec_one - y_t_plus) * (vec_one - pred_t_plus) + y_t_plus * pred_t_plus + 1e-10
            q_t_plus[q_t_plus > 1] = 1.
            z_t_plus = focal_alpha * (-focal_gamma * (1 - q_t_plus) ** (focal_gamma - 1) * np.log(q_t_plus) + \
                                      1/q_t_plus * (1 - q_t_plus) ** focal_gamma) * \
                       (2 * y_t_plus - vec_one) * pred_t_plus * (vec_one - pred_t_plus)
            grad_V_t += norm_time * -(V_t_plus - V_t) + \
                        norm_pred * -(np.dot(np.cos(theta) * z_t + np.sin(theta) * z_t_plus, w1.T) + 
                                      np.sin(theta) * (z_t_plus * tmp_recur * w2.T))
        
        if V_t_minus is not None:
            tmp_recur = np.cos(theta) * V_t + np.sin(theta) * V_t_minus
            grad_V_t += norm_time * (V_t - V_t_minus) + \
                        norm_pred * -(z_t * tmp_recur * w2.T)
        V_t -= lr * grad_V_t
        
        # Compute the gradient w.r.t. w1 & w2
        if V_t_minus is not None:
            tmp_recur = np.cos(theta) * V_t + np.sin(theta) * V_t_minus
            grad_w1 = -norm_pred * np.dot(tmp_recur.T, z_t) + lambda_w * w1
            w1 -= lr * grad_w1
            grad_w2 = -norm_pred * 0.5 * np.dot((tmp_recur**2).T, z_t) + lambda_w * w2
            w2 -= lr * grad_w2
            
        # Compute the gradient w.r.t. we
        grad_we = np.zeros(we.shape)
        for index_attr in range(we.shape[0]):
            tmpA_t = A_ts[index_attr]
            tmp_is, tmp_js, tmp_vs = scipy.sparse.find(tmpA_t)
            tmp = np.mean(np.log(sigmoid(np.sum(V_t[tmp_is,:] * V_t[tmp_js,:], axis=1))) * tmp_vs)
            tmp_grad_sigma_we = sigmoid(we[index_attr,0]) * (1 - sigmoid(we[index_attr,0]))
            grad_we[index_attr,0] = -param_alpha * tmp * tmp_grad_sigma_we
        we -= lr * grad_we
            
        # Compute the gradient w.r.t. theta
        grad_theta = -np.sin(theta) * (np.dot(np.dot(z_t.T, V_t), w1) + \
                                       np.dot(np.dot(z_t.T, tmp_recur * V_t), w2))
        if V_t_minus is not None:
            grad_theta = np.cos(theta) * (np.dot(np.dot(z_t.T, V_t_minus), w1) + \
                                          np.dot(np.dot(z_t.T, tmp_recur * V_t_minus), w2))
        theta -= 0.1 * lr * norm_pred * grad_theta[0][0]
        
        # Compute the gradient w.r.t. V_t_plus
        if V_t_plus is not None:
            tmp_recur = np.cos(theta) * V_t_plus + np.sin(theta) * V_t
            pred_t_plus = sigmoid(np.dot(tmp_recur, w1) + 0.5 * np.dot(tmp_recur**2, w2))
            q_t_plus = (vec_one - y_t_plus) * (vec_one * pred_t_plus) + y_t_plus * pred_t_plus + 1e-10
            q_t_plus[q_t_plus > 1] = 1.
            z_t_plus = focal_alpha * (-focal_gamma * (1 - q_t_plus) ** (focal_gamma - 1) * np.log(q_t_plus) + \
                                      1/q_t_plus * (1 - q_t_plus) ** focal_gamma) * \
                       (2 * y_t_plus - vec_one) * pred_t_plus * (vec_one - pred_t_plus)
            grad_V_t_plus = norm_graph * (-param_alpha * S_t * V_t_plus + R_t * V_t_plus) + \
                            lambda_V * V_t_plus + \
                            norm_laplacian * L_t * L_t * V_t_plus + \
                            norm_time * (V_t_plus - V_t) + \
                            norm_pred * (-z_t_plus * tmp_recur * w2.T)
            V_t_plus -= lr * grad_V_t_plus
        
        # Compute the gradient w.r.t. V_t_minus
        if V_t_minus is not None:
            grad_V_t_minus = norm_graph * (-param_alpha * S_t * V_t_minus + R_t * V_t_minus) + \
                             lambda_V * V_t_minus + \
                             norm_laplacian * L_t * L_t * V_t_minus + \
                             norm_time * (V_t_minus - V_t)
            V_t_minus -= lr * grad_V_t_minus
        
    return date_ym, V_t, w1, w2, we, theta

In [145]:
def validate(input_pkg):
    t, hetergraph = input_pkg
    L_t, A_ts = hetergraph
    date_ym = list_date_seq[t]
    
    key2int_t = dyngraph_info[date_ym]['key2int']
    int2key_t = dyngraph_info[date_ym]['int2key']
    
    y_t = dyngraph_info[date_ym]['y']
    V_t = dyngraph_info[date_ym]['vertex_embed']
    w1 = classifiers['w1']
    w2 = classifiers['w2']
    we = classifiers['we']
    theta = classifiers['theta']
    
    A_t = scipy.sparse.csr_matrix(A_ts[0].shape)
    for index_attr in range(we.shape[0]):
        tmpA_t = A_ts[index_attr]
        A_t += sigmoid(we[index_attr,0]) * tmpA_t
    
    if (t + 1) < seq_length:
        date_ym_plus = list_date_seq[t + 1]
        
        key2int_t_plus = dyngraph_info[date_ym_plus]['key2int']
        int2key_t_plus = dyngraph_info[date_ym_plus]['int2key']
        index_t_2_t_plus = [key2int_t_plus[x] if x in key2int_t_plus else -1 for x in int2key_t]
        
        # Get V_t_plus aligned in V_t index order
        V_t_plus_orig = dyngraph_info[date_ym_plus]['vertex_embed']
        V_t_plus_orig = np.append(V_t_plus_orig, np.zeros((1, V_t_plus_orig.shape[1])), axis=0)
        V_t_plus = V_t_plus_orig[index_t_2_t_plus]
        
        # Get y_t_plus aligned in y_t index order
        y_t_plus_orig = dyngraph_info[date_ym_plus]['y']
        y_t_plus_orig = np.append(y_t_plus_orig, np.zeros((1,1)), axis=0)
        y_t_plus = y_t_plus_orig[index_t_2_t_plus]
    else:
        V_t_plus = None
        y_t_plus = None
        key2int_t_plus = None
        int2key_t_plus = None

    if t > 0:
        date_ym_minus = list_date_seq[t - 1]
        
        key2int_t_minus = dyngraph_info[date_ym_minus]['key2int']
        int2key_t_minus = dyngraph_info[date_ym_minus]['int2key']
        index_t_2_t_minus = [key2int_t_minus[x] if x in key2int_t_minus else -1 for x in int2key_t]
        
        # Get V_t_plus aligned in V_t index order
        V_t_minus_orig = dyngraph_info[date_ym_minus]['vertex_embed']
        V_t_minus_orig = np.append(V_t_minus_orig, np.zeros((1, V_t_minus_orig.shape[1])), axis=0)
        V_t_minus = V_t_minus_orig[index_t_2_t_minus]
                        
        y_t_minus_orig = dyngraph_info[date_ym_minus]['y']
        y_t_minus_orig = np.append(y_t_minus_orig, np.zeros((1,1)), axis=0)
        y_t_minus = y_t_minus_orig[index_t_2_t_minus] 
    else:
        V_t_minus = None
        y_t_minus = None
        key2int_t_minus = None
        int2key_t_minus = None
    
    # Create sparse matrix S
    vec_i, vec_j, vec_v = scipy.sparse.find(A_t)
    vec_data = np.log(sigmoid(np.sum(V_t[vec_i, :] * V_t[vec_j, :], axis=1)))
    
    # Create sparse matrix R
    l = [[u]*L for u in range(A_t.shape[0])]
    smpl_i = [item for sublist in l for item in sublist]
    l = [list(np.random.choice(A_t.shape[0], min(L*A_t[u,:].nnz, A_t.shape[0]), replace=False)) for u in range(A_t.shape[0])]
    smpl_j = [item for sublist in l for item in sublist]
    smpl_index = list(set(zip(smpl_i, smpl_j)) | set(zip(vec_i, vec_j)))
    smpl_i, smpl_j = zip(*smpl_index)
    smpl_data = np.log(sigmoid(-np.sum(V_t[smpl_i, :] * V_t[smpl_j, :], axis=1)))
    
    # Compute normalization term
    norm_graph = 1 / (vec_data.shape[0] + smpl_data.shape[0])
    norm_laplacian = 1 / (V_t.shape[0] * V_t.shape[1])
    norm_time = 1 / (V_t.shape[0] * V_t.shape[1])
    norm_pred = 10 / V_t.shape[0]
    
    loss_t_graph = norm_graph * (-param_alpha * np.sum(vec_v * vec_data) - np.sum(smpl_data)) + \
                   0.5 * lambda_V * np.sum(V_t ** 2)
    
    loss_t_laplacian = 0.5 * norm_laplacian * np.sum((L_t * V_t) ** 2)
    
    loss_t_time = 0
    if V_t_plus is not None:
        loss_t_time += 0.5 * norm_time * np.sum((V_t - V_t_plus) ** 2)
    if V_t_minus is not None:
        loss_t_time += 0.5 * norm_time * np.sum((V_t - V_t_minus) ** 2)
    
    tmp_recur = np.cos(theta) * V_t + np.sin(theta) * V_t_minus if V_t_minus is not None else V_t
    pred_t = sigmoid(np.dot(tmp_recur, w1) + 0.5 * np.dot(tmp_recur**2, w2))
    vec_one = np.ones((y_t.shape[0], 1))
    q_t = (vec_one - y_t) * (vec_one - pred_t) + y_t * pred_t + 1e-10
    q_t[q_t > 1] = 1.
    loss_t_pred = -norm_pred * np.sum(focal_alpha * (vec_one - q_t)**focal_gamma * np.log(q_t))
    
    loss_t = loss_t_graph + loss_t_laplacian + loss_t_time + loss_t_pred
    
    return loss_t, loss_t_graph, loss_t_laplacian, loss_t_time, loss_t_pred

In [146]:
#
# Variables Initialization
# ---------------------------------------------------------------------------------------------------------------
for year, month in list_date_seq:
    num_vertex_t = len(dyngraph_info[(year, month)]['key2int'])
    dyngraph_info[(year, month)]['vertex_embed'] = np.random.rand(num_vertex_t, latent_dim)

classifiers = {'w1': np.random.rand(latent_dim, 1),
               'w2': np.random.rand(latent_dim, 1),
               'we': np.random.rand(len(edge_attrs), 1),
               'theta': np.random.rand()}

#
# Model Training
# ---------------------------------------------------------------------------------------------------------------
list_input_pkg = []
for t in range(len(list_date_seq)):
    list_input_pkg.append((t, 
                           (dyngraph_info[list_date_seq[t]]['graph_laplacian'], 
                            dyngraph_info[list_date_seq[t]]['graph_adjacent'])
                          ))
num_cores = 9
print("Training ......")
for epoch in range(epochs):        
    #                              #
    # Embedding training           #
    # ---------------------------- #
    res_list = []
    with Pool(processes=num_cores) as p:
        max_ = seq_length
        with tqdm_notebook(total=max_) as pbar:
            for input_pkg, res in tqdm_notebook(enumerate(p.imap_unordered(update_params, list_input_pkg))):
                pbar.update()
                res_list.append(res)
    
    date_ym_seq, V_t_seq, w1_seq, w2_seq, we_seq, theta_seq = zip(*res_list)
    for index in range(len(date_ym_seq)):
        date_ym = date_ym_seq[index]
        dyngraph_info[date_ym]['vertex_embed'] = V_t_seq[index]
    
    classifiers['w1'] = np.mean(w1_seq, axis=0)
    classifiers['w2'] = np.mean(w2_seq, axis=0)
    tmp_mul = np.exp(np.mean(we_seq, axis=0))
    classifiers['we'] = tmp_mul / np.sum(tmp_mul)
    classifiers['theta'] = np.mean(theta_seq, axis=0)
    
    #                              #
    # Validation                   #
    # ---------------------------- #
    res_list = []
    with Pool(processes=num_cores) as p:
        max_ = seq_length
        with tqdm_notebook(total=max_) as pbar:
            for input_pkg, res in tqdm_notebook(enumerate(p.imap_unordered(validate, list_input_pkg))):
                pbar.update()
                res_list.append(res)
    
    # Save the checkpoint
    for t in range(seq_length):
        date_ym = list_date_seq[t]
        np.save('../data/vertex_embeddings/{}_heter_superv_recur_focal_logisticMF_embed_{}-{}'.format(
            weight_type, date_ym[0], date_ym[1]), dyngraph_info[(year, month)]['vertex_embed'])  
    
    vec_loss = np.mean(res_list, axis=0)
    degree = (classifiers['theta'] % (2*np.pi)) / (2*np.pi) * 360
    print("Epoch {} Loss => total: {:.4f} , g: {:.4f}, l: {:.4f}, t: {:.4f}, p: {:.4f}, theta: {:.3f}, theta_d: {:.3f}".
          format(epoch, vec_loss[0], vec_loss[1], vec_loss[2], vec_loss[3], vec_loss[4], classifiers['theta'], degree))

Training ......




Epoch 0 Loss => total: 54.6228 , g: 31.1225, l: 0.1683, t: 0.3105, p: 23.0215, theta: 0.865, theta_d: 49.572




Epoch 1 Loss => total: 54.2586 , g: 30.7640, l: 0.1668, t: 0.3064, p: 23.0214, theta: 0.865, theta_d: 49.572




Epoch 2 Loss => total: 53.8999 , g: 30.4113, l: 0.1653, t: 0.3023, p: 23.0210, theta: 0.865, theta_d: 49.565




Epoch 3 Loss => total: 53.5569 , g: 30.0744, l: 0.1639, t: 0.2982, p: 23.0205, theta: 0.865, theta_d: 49.547


...



Epoch 595 Loss => total: 1.0229 , g: 1.0026, l: 0.0017, t: 0.0026, p: 0.0161, theta: 1.118, theta_d: 64.032




Epoch 596 Loss => total: 1.0196 , g: 0.9992, l: 0.0017, t: 0.0026, p: 0.0161, theta: 1.120, theta_d: 64.162




Epoch 597 Loss => total: 1.0191 , g: 0.9987, l: 0.0017, t: 0.0026, p: 0.0162, theta: 1.122, theta_d: 64.292




Epoch 598 Loss => total: 1.0179 , g: 0.9974, l: 0.0017, t: 0.0026, p: 0.0163, theta: 1.124, theta_d: 64.423




Epoch 599 Loss => total: 1.0160 , g: 0.9954, l: 0.0017, t: 0.0025, p: 0.0164, theta: 1.127, theta_d: 64.554
